In [1]:
import pandas as pd
import os
import json
import zipfile
from datetime import datetime
import math

# Import the provided salary data in to Jupyter Notebook
# the salary data csv file path
csv_path = r"C:\Users\jeffr\.cache\kagglehub\datasets\mojtaba142\20112018-salaries-for-san-francisco\versions\1\Total.csv"

# Load the salary data CSV file with try/except error handling
try:
    df = pd.read_csv(csv_path, low_memory=False)
    # Convert salary columns to numeric
    salary_columns = ['BasePay', 'OvertimePay', 'OtherPay', 'Benefits']
    df[salary_columns] = df[salary_columns].apply(pd.to_numeric, errors='coerce')
    print("CSV file loaded successfully.")
    # Display the first 5 rows of the provided salary data table
    print(df.head())
except Exception as e:
    print(f"Error loading CSV: {e}")
    df = None

# Define/create the folder and ZIP paths for the output
folder_path = r"C:\Users\jeffr\Employee Profile"
zip_path = os.path.join(os.path.dirname(folder_path), "Employee Profile.zip")
os.makedirs(folder_path, exist_ok=True)

# Develop a Python function that accepts multiple employee's name as input, and returns their details.
# Function to export mulitiple emplyees profile as CSV and JSON with timestamp & error handling
def export_multiple_employee_profiles(employee_names):
    if df is None:
        print("Cannot proceed: CSV data not loaded.")
        return

    if not employee_names or not isinstance(employee_names, list):
        print("Error: Provide a list of employee names.")
        return

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Create README content. This README is only for the ZIP file content
    readme_text = (
        "Employee Profile ZIP Archive \n"
        f"Generated: {timestamp}\n"
        "Contains individual CSV and JSON files for the following employees:\n"
        + "\n".join(employee_names)
    )

    # Open the ZIP in append mode (overwrite-safe)
    with zipfile.ZipFile(zip_path, 'a', zipfile.ZIP_DEFLATED) as zipf:
        # Add README with time stamp
        readme_filename = f"README_{timestamp}.txt"
        zipf.writestr(readme_filename, readme_text)

        for employee_name in employee_names:
            # Filter employee
            result = df[df['EmployeeName'].str.contains(employee_name, case=False, na=False)]
            if result.empty:
                print(f"No employee found matching'{employee_name}', skipping.")
                continue

            # CSV and JSON file names
            csv_file_name = f"{employee_name.replace(" ", "_")}_Profile_{timestamp}.csv"
            json_file_name = f"{employee_name.replace(" ","_")}_Profile_{timestamp}.json"
            
            # Temporary file paths
            csv_file_path = os.path.join(folder_path, csv_file_name)
            json_file_path = os.path.join(folder_path, json_file_name)

            # Export files locally first
            result.to_csv(csv_file_path, index= False)

            # Convert all Nans to None (nuu in JSON)
            def clean_nan(obj):
                if isinstance(obj, list):
                    return [clean_nan(x) for x in obj]
                elif isinstance(obj, dict):
                    return {k: clean_nan(v) for k, v in obj.items()}
                elif isinstance(obj, float) and math.isnan(obj):
                    return None
                else:
                    return obj

            result_dict = result.to_dict(orient='records')
            result_dict_clean = clean_nan(result_dict)

            with open(json_file_path, 'w') as f:
                json.dump(result_dict_clean, f, indent=4)

            # add files to ZIP
            zipf.write(csv_file_path, arcname=csv_file_name)
            zipf.write(json_file_path, arcname=json_file_name)

            print(f"Added '{employee_name}' profile to ZIP.")

    print(f"All available employee profiles added. ZIP location: {zip_path}")


# Usage
export_multiple_employee_profiles(["Nathaniel", "Gary", "Pardini", "Chong"])
            
            




CSV file loaded successfully.
        EmployeeName                                        JobTitle  \
0     NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY   
1       GARY JIMENEZ                 CAPTAIN III (POLICE DEPARTMENT)   
2     ALBERT PARDINI                 CAPTAIN III (POLICE DEPARTMENT)   
3  CHRISTOPHER CHONG            WIRE ROPE CABLE MAINTENANCE MECHANIC   
4    PATRICK GARDNER    DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)   

     BasePay  OvertimePay   OtherPay  Benefits   TotalPay  TotalPayBenefits  \
0  167411.18         0.00  400184.25       NaN  567595.43         567595.43   
1  155966.02    245131.88  137811.38       NaN  538909.28         538909.28   
2  212739.13    106088.18   16452.60       NaN  335279.91         335279.91   
3   77916.00     56120.71  198306.90       NaN  332343.61         332343.61   
4  134401.60      9737.00  182234.59       NaN  326373.19         326373.19   

   Year  
0  2011  
1  2011  
2  2011  
3  2011  
4  2011  
Ad